In [143]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import sqlite3
import os

In [119]:
# headers = ({'User-Agent':
#             'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
#             'Accept-Language': 'en-US, en;q=0.5'})
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
           "Accept-Encoding":"gzip, deflate", 
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
           "DNT":"1","Connection":"close", 
           "Upgrade-Insecure-Requests":"1"}
main_page_urls = ["https://www.amazon.com/Beyond-Meat-Burger-PARENT-Frozen/dp/B074K5MDLW/"]

In [120]:
def scrape_reviews(main_page_url, headers):
    main_page = requests.get(main_page_url, headers=headers)
    main_page_soup = BeautifulSoup(main_page.content, 'html.parser')
    
    data = []
    
    product_name = main_page_soup.find('span', {'id': 'productTitle'}).get_text().strip()
    brand_name = main_page_soup.find('a', {'id': 'bylineInfo'}).get_text().strip()[7:]
    price = main_page_soup.find('span', {'id': 'priceblock_ourprice'}).get_text().strip()[1:]
    asin = main_page_soup.find('ul', {'class': 'detail-bullet-list'}).findAll('li')[4].findAll('span')[2].get_text()
    overall_rating = main_page_soup.find('span', {'data-hook': 'rating-out-of-text'}).get_text()[:3]
    
    review_url_base = url[:-1].replace("dp", "product-reviews") + "?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber="
    review_urls = [review_url_base + str(i) for i in range(1, 10)]
    
    for review_url in review_urls:
        review_page = requests.get(review_url, headers=headers)
        review_page_soup = BeautifulSoup(review_page.content, 'html.parser')
        username = review_page_soup.findAll('span', {'class': 'a-profile-name'})
        date = review_page_soup.findAll('span', {'data-hook': 'review-date'})
        rating = soup.find_all('i', {'data-hook': 'review-star-rating'})
        review = soup.find_all('span', {'data-hook': 'review-body'})
        
        for i in range(2, 8):
            try:
                reviewer_username = username[i].get_text()
                review_date = date[i].get_text()[33:]
                review_rating = rating[i].span.get_text()[:3]
                review_text = review[i].get_text()[:-9].strip()

                data.append([brand_name, product_name, asin, price, overall_rating, reviewer_username, review_date, review_rating, review_text])
                
            except:
                continue
    return data

In [144]:
def write_to_db(data):
    conn = sqlite3.connect("reviewdb.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS reviews('brand_name', 'product_name', 'asin', 'price', 'overall_rating', 'reviewer_username', 'review_date', 'review_rating', 'review_text')""")
    
    for entry in data:
        c.execute("INSERT INTO reviews ('brand_name', 'product_name', 'asin', 'price', 'overall_rating', 'reviewer_username', 'review_date', 'review_rating', 'review_text') VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                  (entry[0], entry[1], entry[2], entry[3], entry[4], entry[5], entry[6], entry[7], entry[8]))
        
    conn.commit()
    conn.close()

In [147]:
def write_to_csv(data):
    asin = data[0][2]
    file_name = asin + "Reviews.csv"

    with open(file_name, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data)

    f.close()

In [148]:
for main_page_url in main_page_urls:
    data = scrape_reviews(main_page_url, headers)
    write_to_db(data)
    write_to_csv(data)

UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f319' in position 96: character maps to <undefined>

In [ ]:
conn = sqlite3.connect("reviewdb.db")
c = conn.cursor()
c.execute("SELECT * FROM reviews")
output = c.fetchall()
conn.close()
print(output)